In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.chdir(globals()['_dh'][0])
os.chdir('..')
print(os.path.abspath(os.curdir))

In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random 
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
from scipy import signal as sg
# from sklearn.utils import shuffle

import rfcutils # this corresponds to utility functions provided for the challenge

from src import unet_model as unet
from src.time_proc import long_window

get_sinr = lambda s, i: 10*np.log10(np.mean(np.abs(s)**2)/np.mean(np.abs(i)**2))
get_pow = lambda s: np.mean(np.abs(s)**2)

2022-08-17 10:15:56.432757: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


---

In [3]:
def eval_qpsk_demod(sig_est, sig_true, start_idx):
    qpsk_sig_idx = 16 - start_idx%16
    bit_est = rfcutils.matched_filter_demod(sig_est[qpsk_sig_idx:])
    bit_true = rfcutils.matched_filter_demod(sig_true[qpsk_sig_idx:])
    ber = np.sum(bit_est != bit_true)/len(bit_true)
    assert len(bit_est) == len(bit_true)
    return ber

def eval_sigs(sig_est, sig_true, start_idx):
    mse = get_pow(sig_est - sig_true)
    mse_db = 10*np.log10(mse)
    sdr = get_sinr(sig_true, sig_est-sig_true)
    ber = eval_qpsk_demod(sig_est, sig_true, start_idx)
    return (mse, mse_db, sdr, ber)

## Load Data

In [4]:
all_sig_mixture, all_sig1, all_sig2, all_sync_idx2, all_sig1_syms, all_sig1_bits, all_start_idx2 = pickle.load(open('dataset/Ex3C3_QPSK_OFDMQAM16.pickle','rb'))
all_val_sig_mixture, all_val_sig1, all_val_sig2, all_val_sync_idx2, all_val_sig1_syms, all_val_sig1_bits, all_val_start_idx2 = pickle.load(open('dataset/Ex3C3_ValSet_QPSK_OFDMQAM16.pickle','rb'))


## Parameters

In [5]:
all_sinr = np.arange(-30, 4, 1.5)
print(len(all_sinr))
n_per_sinr_tr = 1000
n_per_sinr_val = 100
training_examples = 400
val_examples = 100
seq_len = 10240
window_len = 10240
eff_train_ex = int(training_examples*seq_len/window_len)

23


## Process data

In [6]:
all_sig1_tr_val = []
all_mixture_tr_val = []

for j in tqdm(np.arange(len(all_sinr))):
    for i in range(training_examples):
    
        all_sig1_ex = np.array(all_sig1[j*n_per_sinr_tr + i])
        all_sig1_ex_w = all_sig1_ex[:seq_len]
        all_sig2_ex = np.array(all_sig2[j*n_per_sinr_tr + i])
        all_sig2_ex_w = all_sig2_ex[:seq_len]
        all_sig_mixture_ex = np.array(all_sig_mixture[j*n_per_sinr_tr + i])
        all_sig_mixture_ex_w = all_sig_mixture_ex[:seq_len]
        
        all_sig1_tr_val.append(all_sig1_ex_w)
        all_mixture_tr_val.append(all_sig_mixture_ex_w)
        
        coeff = np.sqrt(np.mean(np.abs(all_sig1_ex_w)**2)/(np.mean(np.abs(all_sig2_ex_w)**2)))
  
for j in tqdm(np.arange(len(all_sinr))):
    for i in range(val_examples):

        all_sig1_ex = np.array(all_val_sig1[j*n_per_sinr_val + i])
        all_sig1_ex_w = all_sig1_ex[:seq_len]
        all_sig2_ex = np.array(all_val_sig2[j*n_per_sinr_val + i])
        all_sig2_ex_w = all_sig2_ex[:seq_len]
        all_sig_mixture_ex = np.array(all_val_sig_mixture[j*n_per_sinr_val + i])
        all_sig_mixture_ex_w = all_sig_mixture_ex[:seq_len]
        
        all_sig1_tr_val.append(all_sig1_ex_w)
        all_mixture_tr_val.append(all_sig_mixture_ex_w)
        
        coeff = np.sqrt(np.mean(np.abs(all_sig1_ex_w)**2)/(np.mean(np.abs(all_sig2_ex_w)**2)))
    
all_sig1_tr_val = np.array(all_sig1_tr_val)
all_mixture_tr_val = np.array(all_mixture_tr_val)

100%|██████████| 23/23 [00:00<00:00, 24.39it/s]


## Train

In [ ]:
sig1_out = all_sig1_tr_val.reshape(-1,window_len)

out1_comp = np.dstack((sig1_out.real, sig1_out.imag))

sig_mixture_out = all_mixture_tr_val.reshape(-1,window_len)
mixture_bands_comp = np.dstack((sig_mixture_out.real, sig_mixture_out.imag))

print(f'Output shape: {out1_comp.shape}; Input shape: {mixture_bands_comp.shape}')

long_k_sz = 101
model_name = f'qpsk_ofdm64_W{window_len}_TS{training_examples}_K{long_k_sz}'
print(f'Training {model_name}')
nn_model = unet.get_unet_model_XL_2((window_len, 2), k_sz=3, long_k_sz=long_k_sz, start_idx=0, window_len=window_len)
# nn_model.load_weights('trained_models/ofdm_40960_K101_XL_TS1000_e300_maskBER05_sync_4in')
checkpoint_filepath = f'./tmp_checkpoints/checkpoint_qpsk_ofdm64_W{window_len}_TS{training_examples}_K{long_k_sz}'
IR_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)
nn_model.fit(mixture_bands_comp, out1_comp, epochs=2000, batch_size=32, validation_split=0.2,shuffle=True, verbose=1,callbacks=[stop_early,IR_model_checkpoint_callback])

Output shape: (11500, 10240, 2); Input shape: (11500, 10240, 2)
Training qpsk_ofdm64_W10240_TS400_K101


2022-08-17 10:16:34.420503: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-17 10:16:34.421596: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-17 10:16:34.516847: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-08-17 10:16:34.516919: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-08-17 10:16:34.521612: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-08-17 10:16:34.521668: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2

Epoch 1/2000


2022-08-17 10:16:37.931336: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-08-17 10:16:39.680467: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-08-17 10:16:40.184082: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-08-17 10:16:48.251165: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-08-17 10:16:48.615500: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


287/288 [============================>.] - ETA: 0s - loss: 0.3899